In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

print("Cargando datos...")

# 📌 Cargar dataset y tomar una muestra del 30%
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet').sample(frac=0.30, random_state=42)

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['family', 'cod_section_group', 'device_type', 'color_id', 'hour']
for col in categorical_columns:
    train[col] = train[col].astype('category')

# 📌 Definir características y etiqueta
features = [col for col in train.columns if col not in ["session_id", "add_to_cart"]]
target = "add_to_cart"

print("Preparando folds de validación...")

# 📌 Usar GroupKFold para evitar fuga entre sesiones
group_kfold = GroupKFold(n_splits=2)
folds = list(group_kfold.split(train[features], train[target], groups=train["session_id"]))

# 📌 Definir las características categóricas
categorical_features = ['family', 'country', 'pagetype_group', 'cod_section_group', 'device_type', 'color_id', 'hour']

# 📌 Calcular scale_pos_weight para manejar desbalanceo de clases
neg_count = train[target].value_counts()[0]
pos_count = train[target].value_counts()[1]
scale_pos_weight = neg_count / pos_count

# 📌 Definir parámetros del modelo con mayor regularización
params = {
    "objective": "lambdarank",
    "boosting_type": "gbdt",
    "metric": ["ndcg", "map"],
    "learning_rate": 0.02,
    "min_data_in_leaf": 200,
    "num_leaves": 255,
    "max_depth": 9,
    "verbosity": -1,
    "lambdarank_truncation_level": 15,
    "feature_fraction": 0.7,
    "bagging_fraction": 0.8,
    "bagging_freq": 10,
    "lambda_l1": 35.0,    # Incrementado de 5.0 a 20.0
    "lambda_l2": 70.0,    # Incrementado de 15.0 a 50.0
    "lambdarank_norm": True,
    "label_gain": [i for i in range(25)],
    "seed": 42,
    "eval_at": [5],
    "num_threads": 0,
    "scale_pos_weight": scale_pos_weight,  # Manejar desbalanceo
}

print("Entrenando el modelo con validación cruzada...\n")

# 🚀 Entrenar usando GroupKFold
models = []
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Fold {fold + 1}/{len(folds)}")
    X_train = train.iloc[train_idx][features]
    y_train = train.iloc[train_idx][target]
    session_train = train.iloc[train_idx]["session_id"]
    X_val = train.iloc[val_idx][features]
    y_val = train.iloc[val_idx][target]
    session_val = train.iloc[val_idx]["session_id"]

    # 📌 Calcular `query` correctamente
    query_train = session_train.value_counts().sort_index().tolist()
    query_val = session_val.value_counts().sort_index().tolist()

    # 📌 Crear datasets de LightGBM con `group` y características categóricas
    lgb_train = lgb.Dataset(X_train, label=y_train, group=query_train, categorical_feature=categorical_features, free_raw_data=False)
    lgb_val = lgb.Dataset(X_val, label=y_val, group=query_val, reference=lgb_train, categorical_feature=categorical_features, free_raw_data=False)

    # 🚀 Entrenar el modelo con early stopping
    callbacks = [lgb.early_stopping(stopping_rounds=30, first_metric_only=True, verbose=True)]
    
    model = lgb.train(params, lgb_train, valid_sets=[lgb_val], num_boost_round=2000, callbacks=callbacks)
    models.append(model)
    
    print("\n")

# 📥 Guardar el último modelo entrenado
models[-1].save_model('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_last_try.txt')
print("📥 Modelo guardado")

Cargando datos...
Preparando folds de validación...
Entrenando el modelo con validación cruzada...

Fold 1/2
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[807]	valid_0's ndcg@5: 0.934851	valid_0's map@5: 0.921967
Evaluated only: ndcg@5


Fold 2/2
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[976]	valid_0's ndcg@5: 0.93486	valid_0's map@5: 0.921957
Evaluated only: ndcg@5


📥 Modelo guardado


In [ ]:
from sklearn.metrics import ndcg_score, average_precision_score
import numpy as np
import pandas as pd


# 📌 Restaurar `session_id` y `partnumber` en `X_val`
X_val["session_id"] = session_val.values  
X_val["partnumber"] = train.loc[X_val.index, "partnumber"]  # Recuperamos el producto real

# 📌 Crear `y_val_df` asegurando que `session_id` y `partnumber` estén presentes
y_val_df = pd.DataFrame({
    "session_id": np.ravel(session_val),  
    "partnumber": train.loc[y_val.index, "partnumber"],  # Recuperamos productos comprados
    "add_to_cart": np.ravel(y_val)
})

# 📌 Obtener las características utilizadas en el entrenamiento
features = model.feature_name()

# 📌 Asegurar que `X_val` tenga exactamente las mismas columnas antes de predecir
X_val_pred = X_val[features]

# 📌 Generar predicciones en el conjunto de validación
X_val["score"] = model.predict(X_val_pred)

# 📌 Obtener recomendaciones por sesión (productos en lugar de índices)
session_recommendations = (
    X_val.sort_values(["session_id", "score"], ascending=[True, False])
    .groupby("session_id")["partnumber"]
    .apply(list)  # Ahora obtenemos productos en lugar de índices
    .to_dict()
)

# 📌 Obtener ground truth (productos realmente añadidos al carrito)
true_items = (
    y_val_df[y_val_df["add_to_cart"] == 1]
    .groupby("session_id")["partnumber"]
    .apply(list)
    .to_dict()
)

# 📌 Función de evaluación corregida
def evaluate_ranking(predictions, ground_truth, k=5):
    ndcg_scores, map_scores, mrr_scores, hit_rates = [], [], [], []

    for session_id, recommended_items in predictions.items():
        true_list = ground_truth.get(session_id, [])

        # 📌 Filtrar sesiones sin suficientes elementos para evaluar NDCGa
        if len(true_list) < 2 or len(recommended_items) < 2:
            continue  

        # 📌 Asegurar que y_true es 2D con valores de relevancia binarios
        y_true = np.array([[(1 if i in true_list else 0) for i in recommended_items[:k]]])
        y_score = np.array([[k - i for i in range(min(k, len(recommended_items)))]])

        # 📌 Calcular métricas solo si hay al menos un relevante
        if np.sum(y_true) > 0:
            ndcg_scores.append(ndcg_score(y_true, y_score))
            map_scores.append(average_precision_score(y_true[0], y_score[0]))
            mrr_scores.append(next((1 / (i + 1) for i, val in enumerate(y_true[0]) if val == 1), 0))
            hit_rates.append(1)
        else:
            ndcg_scores.append(0)
            map_scores.append(0)
            mrr_scores.append(0)
            hit_rates.append(0)

    return {
        "NDCG@5": np.mean(ndcg_scores) if ndcg_scores else 0,
        "MAP@5": np.mean(map_scores) if map_scores else 0,
        "MRR": np.mean(mrr_scores) if mrr_scores else 0,
        "Hit Rate @5": np.mean(hit_rates) if hit_rates else 0,
    }

# 📌 Evaluar el modelo
metrics = evaluate_ranking(session_recommendations, true_items)
print("\n📊 Métricas de Evaluación:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


In [3]:
# predictions_with_embeddings_backup_v3_Item2Vec -- 253/900
import json
import pandas as pd
from collections import defaultdict
import lightgbm as lgb
from gensim.models import Word2Vec
import numpy as np

print("Cargando modelo y datos...")

# 📤 Cargar el modelo guardado
model = lgb.Booster(model_file='/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_last_try.txt')

# 📌 Cargar datasets
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet')
test = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/test_final_aligned.parquet')

# 📌 Definir las características como en el entrenamiento
features = [col for col in train.columns if col not in ["session_id", "add_to_cart"]]

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['family', 'cod_section_group', 'device_type', 'color_id', 'hour']
for col in categorical_columns:
    test[col] = test[col].astype('category')

# 📌 Asegurar que las columnas de test están en el mismo orden que en el entrenamiento
test = test[["session_id"] + features]

print("Realizando predicciones...")

# 📌 Predecir probabilidades en test
test["score"] = model.predict(test[features])

# 📌 Generar primeras recomendaciones por sesión
session_recommendations = (
    test.groupby("session_id", group_keys=False)
    .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())
    .to_dict()
)

print("Entrenando modelo Item2Vec...")

# 📌 Obtener secuencias de productos por sesión en el conjunto de entrenamiento
train_sessions = train.groupby('session_id')['partnumber'].apply(list)

# Convertir los IDs de productos a strings (Word2Vec requiere strings)
train_sessions = train_sessions.apply(lambda x: [str(item) for item in x])

# Lista de secuencias para entrenar el modelo
sentences = train_sessions.tolist()

# 📌 Entrenar el modelo Word2Vec

model_i2v = Word2Vec(
    sentences,
    vector_size=110,        # Ajuste de vector_size
    window=8,              # Ajuste de window
    min_count=2,            # Ajuste de min_count
    workers=8,
    sg=0,                   # Uso de Skip-Gram // 0 CBOW 
    negative=20,            # Muestreo negativo +
    epochs=60,              # Aumento de épocas +
    sample=1e-4,            # +     
    # min_alpha=0.0001,      # Tasa de aprendizaje mínima       
)

# Objeto para obtener los embeddings de productos
product_embeddings = model_i2v.wv

print("Calculando artículos populares...")

# 📌 Calcular los artículos más populares como último recurso
popular_items = train.loc[train['add_to_cart'] == 1, 'partnumber'].value_counts().index.tolist()

print("Generando recomendaciones finales usando Item2Vec...")

# Número de productos similares a obtener
top_n_similar = 5

final_recommendations = {}

for session_id, items in session_recommendations.items():
    unique_items = []
    seen_items = set()
    
    # Añadir items del modelo asegurando unicidad
    for item in items:
        if item not in seen_items:
            unique_items.append(item)
            seen_items.add(item)
        if len(unique_items) == 5:
            break
    
    # Si no tenemos suficientes, buscar artículos similares con Item2Vec
    if len(unique_items) < 5:
        for item in items:
            similar_items = []
            try:
                # Obtener similares con Item2Vec
                similar_items = product_embeddings.most_similar(str(item), topn=top_n_similar)
                # Convertir los IDs de productos a enteros
                similar_items = [int(sim_item[0]) for sim_item in similar_items]
            except KeyError:
                # Si el producto no está en el vocabulario
                continue
            for sim_item in similar_items:
                if sim_item not in seen_items:
                    unique_items.append(sim_item)
                    seen_items.add(sim_item)
                if len(unique_items) == 5:
                    break
            if len(unique_items) == 5:
                break
    
    # Si aún no tenemos suficientes, completar con artículos populares
    if len(unique_items) < 5:
        for item in popular_items:
            if item not in seen_items:
                unique_items.append(item)
                seen_items.add(item)
            if len(unique_items) == 5:
                break

    final_recommendations[session_id] = unique_items

print("Guardando recomendaciones en JSON...")

# 📌 Guardar en JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_last_try.json"
with open(output_path, "w") as f:
    json.dump({"target": final_recommendations}, f, indent=4)

print(f"\n✅ Archivo de predicciones guardado en: {output_path}")

Cargando modelo y datos...
Realizando predicciones...


/tmp/ipykernel_525577/2297624147.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())


Entrenando modelo Item2Vec...
Calculando artículos populares...
Generando recomendaciones finales usando Item2Vec...
Guardando recomendaciones en JSON...

✅ Archivo de predicciones guardado en: /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_last_try.json


---

In [6]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

print("Cargando datos...")

# 📌 Cargar dataset y tomar una muestra del 50%
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet').sample(frac=0.30, random_state=42)

# 📌 Seleccionar solo las variables más importantes
important_features = [
    'country', 
    'device_type', 
    'pagetype_group', 
    'cod_section_group', 
    'family', 
    'color_id', 
    'hour', 
    'user_id', 
    'partnumber', 
    'RFM_PCA'
]

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['country', 'device_type', 'pagetype_group', 'cod_section_group', 'family', 'color_id', 'hour']
for col in categorical_columns:
    train[col] = train[col].astype('category')

# 📌 Definir características y etiqueta
features = important_features  # Usar solo las variables importantes
target = "add_to_cart"

print("Preparando folds de validación...")

# 📌 Usar GroupKFold para evitar fuga entre sesiones
group_kfold = GroupKFold(n_splits=2)
folds = list(group_kfold.split(train[features], train[target], groups=train["session_id"]))

# 📌 Definir las características categóricas
categorical_features = ['country', 'device_type', 'pagetype_group', 'cod_section_group', 'family', 'color_id', 'hour']

# 📌 Calcular scale_pos_weight para manejar desbalanceo de clases
neg_count = train[target].value_counts()[0]
pos_count = train[target].value_counts()[1]
scale_pos_weight = neg_count / pos_count

# 📌 Definir parámetros del modelo con mayor regularización
params = {
    "objective": "lambdarank",
    "boosting_type": "gbdt",
    "metric": ["ndcg", "map"],
    "learning_rate": 0.01,
    "min_data_in_leaf": 200,
    "num_leaves": 255,
    "max_depth": 9,
    "verbosity": -1,
    "lambdarank_truncation_level": 15,
    "feature_fraction": 0.7,
    "bagging_fraction": 0.8,
    "bagging_freq": 10,
    "lambda_l1": 35.0,    # Incrementado de 5.0 a 35.0
    "lambda_l2": 70.0,    # Incrementado de 15.0 a 70.0
    "lambdarank_norm": True,
    "label_gain": [i for i in range(25)],
    "seed": 42,
    "eval_at": [5],
    "num_threads": 0,
    "scale_pos_weight": scale_pos_weight,  # Manejar desbalanceo
}

print("Entrenando el modelo con validación cruzada...\n")

# 🚀 Entrenar usando GroupKFold
models = []
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Fold {fold + 1}/{len(folds)}")
    X_train = train.iloc[train_idx][features]
    y_train = train.iloc[train_idx][target]
    session_train = train.iloc[train_idx]["session_id"]
    X_val = train.iloc[val_idx][features]
    y_val = train.iloc[val_idx][target]
    session_val = train.iloc[val_idx]["session_id"]

    # 📌 Calcular `query` correctamente
    query_train = session_train.value_counts().sort_index().tolist()
    query_val = session_val.value_counts().sort_index().tolist()

    # 📌 Crear datasets de LightGBM con `group` y características categóricas
    lgb_train = lgb.Dataset(X_train, label=y_train, group=query_train, categorical_feature=categorical_features, free_raw_data=False)
    lgb_val = lgb.Dataset(X_val, label=y_val, group=query_val, reference=lgb_train, categorical_feature=categorical_features, free_raw_data=False)

    # 🚀 Entrenar el modelo con early stopping
    callbacks = [lgb.early_stopping(stopping_rounds=30, first_metric_only=True, verbose=True)]
    
    model = lgb.train(params, lgb_train, valid_sets=[lgb_val], num_boost_round=2000, callbacks=callbacks)
    models.append(model)
    
    print("\n")

# 📥 Guardar el último modelo entrenado
models[-1].save_model('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_last_try_v2.txt')
print("📥 Modelo guardado")


Cargando datos...
Preparando folds de validación...
Entrenando el modelo con validación cruzada...

Fold 1/2
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[58]	valid_0's ndcg@5: 0.933622	valid_0's map@5: 0.920671
Evaluated only: ndcg@5


Fold 2/2
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[222]	valid_0's ndcg@5: 0.933661	valid_0's map@5: 0.92069
Evaluated only: ndcg@5


📥 Modelo guardado


In [5]:
import json
import pandas as pd
from collections import defaultdict
import lightgbm as lgb
from gensim.models import Word2Vec
import numpy as np

print("Cargando modelo y datos...")

# 📤 Cargar el modelo guardado
model = lgb.Booster(model_file='/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_last_try_v2.txt')

# 📌 Cargar datasets
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet')
test = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/test_final_aligned.parquet')

# 📌 Seleccionar solo las variables más importantes
important_features = [
    'country', 
    'device_type', 
    'pagetype_group', 
    'cod_section_group', 
    'family', 
    'color_id', 
    'hour', 
    'user_id', 
    'partnumber', 
    'RFM_PCA'
]

# 📌 Definir las características como en el entrenamiento
features = important_features  # Usar solo las variables importantes

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['country', 'device_type', 'pagetype_group', 'cod_section_group', 'family', 'color_id', 'hour']
for col in categorical_columns:
    test[col] = test[col].astype('category')

# 📌 Asegurar que las columnas de test están en el mismo orden que en el entrenamiento
test = test[["session_id"] + features]

print("Realizando predicciones...")

# 📌 Predecir probabilidades en test
test["score"] = model.predict(test[features])

# 📌 Generar primeras recomendaciones por sesión
session_recommendations = (
    test.groupby("session_id", group_keys=False)
    .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())
    .to_dict()
)

print("Entrenando modelo Item2Vec...")

# 📌 Obtener secuencias de productos por sesión en el conjunto de entrenamiento
train_sessions = train.groupby('session_id')['partnumber'].apply(list)

# Convertir los IDs de productos a strings (Word2Vec requiere strings)
train_sessions = train_sessions.apply(lambda x: [str(item) for item in x])

# Lista de secuencias para entrenar el modelo
sentences = train_sessions.tolist()

# 📌 Entrenar el modelo Word2Vec
model_i2v = Word2Vec(
    sentences,
    vector_size=110,        # Ajuste de vector_size
    window=8,               # Ajuste de window
    min_count=2,            # Ajuste de min_count
    workers=10,
    sg=0,                   # Uso de CBOW
    negative=20,            # Muestreo negativo aumentado
    epochs=60,              # Aumento de épocas
    sample=1e-4,            # Muestreo
    # min_alpha=0.0001,      # Tasa de aprendizaje mínima       
)

# Objeto para obtener los embeddings de productos
product_embeddings = model_i2v.wv

print("Calculando artículos populares...")

# 📌 Calcular los artículos más populares como último recurso
popular_items = train.loc[train['add_to_cart'] == 1, 'partnumber'].value_counts().index.tolist()

print("Generando recomendaciones finales usando Item2Vec...")

# Número de productos similares a obtener
top_n_similar = 5

final_recommendations = {}

for session_id, items in session_recommendations.items():
    unique_items = []
    seen_items = set()
    
    # Añadir items del modelo asegurando unicidad
    for item in items:
        if item not in seen_items:
            unique_items.append(item)
            seen_items.add(item)
        if len(unique_items) == 5:
            break
    
    # Si no tenemos suficientes, buscar artículos similares con Item2Vec
    if len(unique_items) < 5:
        for item in items:
            similar_items = []
            try:
                # Obtener similares con Item2Vec
                similar_items = product_embeddings.most_similar(str(item), topn=top_n_similar)
                # Convertir los IDs de productos a enteros
                similar_items = [int(sim_item[0]) for sim_item in similar_items]
            except KeyError:
                # Si el producto no está en el vocabulario
                continue
            for sim_item in similar_items:
                if sim_item not in seen_items:
                    unique_items.append(sim_item)
                    seen_items.add(sim_item)
                if len(unique_items) == 5:
                    break
            if len(unique_items) == 5:
                break
    
    # Si aún no tenemos suficientes, completar con artículos populares
    if len(unique_items) < 5:
        for item in popular_items:
            if item not in seen_items:
                unique_items.append(item)
                seen_items.add(item)
            if len(unique_items) == 5:
                break

    final_recommendations[session_id] = unique_items

print("Guardando recomendaciones en JSON...")

# 📌 Guardar en JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_last_try_v2.json"
with open(output_path, "w") as f:
    json.dump({"target": final_recommendations}, f, indent=4)

print(f"\n✅ Archivo de predicciones guardado en: {output_path}")


Cargando modelo y datos...
Realizando predicciones...


/tmp/ipykernel_525577/21717755.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())


Entrenando modelo Item2Vec...
Calculando artículos populares...
Generando recomendaciones finales usando Item2Vec...
Guardando recomendaciones en JSON...

✅ Archivo de predicciones guardado en: /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_last_try_v2.json
